## 1. Import Libraries

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from math import pi
import warnings
warnings.filterwarnings('ignore')

# Set plot style
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")

print("✅ Libraries imported!")

## 2. Load All Model Results

In [ ]:
# Load classical ML results
try:
    classical_results = pd.read_csv('../results/classical_ml_results.csv')
    print("✅ Classical ML results loaded")
except:
    # If file doesn't exist, use example data
    classical_results = pd.DataFrame({
        'Model': ['Logistic Regression', 'Naive Bayes', 'SVM', 'Random Forest'],
        'Accuracy': [0.88, 0.85, 0.89, 0.86],
        'Precision': [0.87, 0.83, 0.88, 0.85],
        'Recall': [0.89, 0.87, 0.90, 0.87],
        'F1-Score': [0.88, 0.85, 0.89, 0.86]
    })
    print("⚠️ Using example classical ML data")

# Load deep learning results
try:
    dl_results = pd.read_csv('../results/deep_learning_results.csv')
    print("✅ Deep learning results loaded")
except:
    # If file doesn't exist, use example data
    dl_results = pd.DataFrame({
        'Model': ['LSTM', 'DistilBERT'],
        'Accuracy': [0.87, 0.92],
        'Precision': [0.86, 0.91],
        'Recall': [0.88, 0.93],
        'F1-Score': [0.87, 0.92]
    })
    print("⚠️ Using example deep learning data")

# Combine all results
all_results = pd.concat([classical_results, dl_results], ignore_index=True)
all_results['Type'] = ['Classical', 'Classical', 'Classical', 'Classical', 'Deep Learning', 'Deep Learning']

print("\n📊 ALL MODEL RESULTS:")
print("=" * 80)
print(all_results.to_string(index=False))
print("=" * 80)

## 3. Comprehensive Model Comparison

### 3.1 Performance Metrics Comparison

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(18, 14))

# Plot 1: Grouped Bar Chart - All Metrics
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score']
x = np.arange(len(all_results))
width = 0.2

for i, metric in enumerate(metrics):
    axes[0, 0].bar(x + i*width - 1.5*width, all_results[metric], width, 
                   label=metric, alpha=0.8, edgecolor='black')

axes[0, 0].set_xlabel('Model', fontsize=12, fontweight='bold')
axes[0, 0].set_ylabel('Score', fontsize=12, fontweight='bold')
axes[0, 0].set_title('All Models - Performance Metrics Comparison', fontsize=14, fontweight='bold')
axes[0, 0].set_xticks(x)
axes[0, 0].set_xticklabels(all_results['Model'], rotation=30, ha='right', fontsize=9)
axes[0, 0].legend(fontsize=10)
axes[0, 0].grid(axis='y', alpha=0.3)
axes[0, 0].set_ylim(0.8, 1.0)

# Plot 2: Heatmap
metrics_data = all_results[metrics].values
sns.heatmap(metrics_data.T, annot=True, fmt='.3f', cmap='YlGnBu',
            xticklabels=all_results['Model'], yticklabels=metrics,
            ax=axes[0, 1], cbar_kws={'label': 'Score'}, vmin=0.8, vmax=1.0)
axes[0, 1].set_title('Performance Heatmap - All Models', fontsize=14, fontweight='bold')
axes[0, 1].set_xticklabels(axes[0, 1].get_xticklabels(), rotation=30, ha='right', fontsize=9)

# Plot 3: Classical vs Deep Learning
type_comparison = all_results.groupby('Type')[metrics].mean()
type_comparison.plot(kind='bar', ax=axes[1, 0], width=0.7, alpha=0.8, edgecolor='black')
axes[1, 0].set_title('Classical ML vs Deep Learning - Average Performance', 
                      fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Model Type', fontsize=12, fontweight='bold')
axes[1, 0].set_ylabel('Average Score', fontsize=12, fontweight='bold')
axes[1, 0].set_xticklabels(axes[1, 0].get_xticklabels(), rotation=0)
axes[1, 0].legend(fontsize=10, loc='lower right')
axes[1, 0].grid(axis='y', alpha=0.3)
axes[1, 0].set_ylim(0.8, 1.0)

# Plot 4: Radar Chart - Best Models
ax = plt.subplot(224, projection='polar')
categories = metrics
N = len(categories)
angles = [n / float(N) * 2 * pi for n in range(N)]
angles += angles[:1]

# Get best models from each type
best_classical = all_results[all_results['Type'] == 'Classical'].iloc[
    all_results[all_results['Type'] == 'Classical']['F1-Score'].idxmax()
]
best_dl = all_results[all_results['Type'] == 'Deep Learning'].iloc[
    all_results[all_results['Type'] == 'Deep Learning']['F1-Score'].idxmax() - 4
]

for model_data, label, color in [(best_classical, f'Best Classical: {best_classical["Model"]}', '#ff7f0e'),
                                  (best_dl, f'Best DL: {best_dl["Model"]}', '#2ca02c')]:
    values = model_data[metrics].values.tolist()
    values += values[:1]
    ax.plot(angles, values, 'o-', linewidth=2, label=label, color=color)
    ax.fill(angles, values, alpha=0.15, color=color)

ax.set_xticks(angles[:-1])
ax.set_xticklabels(categories, fontsize=10)
ax.set_ylim(0.8, 1.0)
ax.set_title('Best Performing Models - Detailed Comparison', 
             fontsize=13, fontweight='bold', pad=20)
ax.legend(loc='upper right', fontsize=9)
ax.grid(True)

plt.tight_layout()
plt.savefig('../results/figures/comprehensive_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("✅ Comprehensive comparison visualization created!")

### 3.2 Model Rankings

In [ ]:
# Rank models by F1-Score
ranked = all_results.sort_values('F1-Score', ascending=False).reset_index(drop=True)
ranked['Rank'] = range(1, len(ranked) + 1)

print("\n🏆 MODEL RANKINGS (by F1-Score):")
print("=" * 90)
print(ranked[['Rank', 'Model', 'Type', 'Accuracy', 'Precision', 'Recall', 'F1-Score']].to_string(index=False))
print("=" * 90)

# Visualize rankings
plt.figure(figsize=(12, 6))
colors = ['#2ca02c' if t == 'Deep Learning' else '#ff7f0e' for t in ranked['Type']]
bars = plt.barh(ranked['Model'], ranked['F1-Score'], color=colors, alpha=0.8, edgecolor='black')

# Add value labels
for i, (model, score) in enumerate(zip(ranked['Model'], ranked['F1-Score'])):
    plt.text(score + 0.005, i, f'{score:.3f}', va='center', fontweight='bold', fontsize=10)

plt.xlabel('F1-Score', fontsize=12, fontweight='bold')
plt.title('Model Rankings by F1-Score', fontsize=14, fontweight='bold')
plt.xlim(0.8, 1.0)
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

# Add legend
from matplotlib.patches import Patch
legend_elements = [
    Patch(facecolor='#2ca02c', alpha=0.8, edgecolor='black', label='Deep Learning'),
    Patch(facecolor='#ff7f0e', alpha=0.8, edgecolor='black', label='Classical ML')
]
plt.legend(handles=legend_elements, loc='lower right', fontsize=11)

plt.tight_layout()
plt.savefig('../results/figures/model_rankings.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Rankings visualization created!")

## 4. Key Insights and Analysis

### 4.1 Automated Insights

In [ ]:
print("\n" + "="*90)
print("KEY INSIGHTS FROM MODEL COMPARISON")
print("="*90)

# Best overall model
best_model = ranked.iloc[0]
print(f"\n1️⃣ BEST OVERALL MODEL:")
print(f"   Model: {best_model['Model']}")
print(f"   Type: {best_model['Type']}")
print(f"   F1-Score: {best_model['F1-Score']:.4f}")
print(f"   Accuracy: {best_model['Accuracy']:.4f}")

# Best classical model
best_classical = all_results[all_results['Type'] == 'Classical'].sort_values('F1-Score', ascending=False).iloc[0]
print(f"\n2️⃣ BEST CLASSICAL ML MODEL:")
print(f"   Model: {best_classical['Model']}")
print(f"   F1-Score: {best_classical['F1-Score']:.4f}")
print(f"   Strength: Fast inference, interpretable")

# Performance gap
performance_gap = best_model['F1-Score'] - best_classical['F1-Score']
print(f"\n3️⃣ DEEP LEARNING ADVANTAGE:")
print(f"   Performance gap: {performance_gap:.4f} ({performance_gap*100:.2f}% improvement)")
print(f"   Trade-off: Higher accuracy vs longer training time")

# Model diversity
print(f"\n4️⃣ MODEL PERFORMANCE RANGE:")
print(f"   Highest F1: {all_results['F1-Score'].max():.4f}")
print(f"   Lowest F1: {all_results['F1-Score'].min():.4f}")
print(f"   Range: {all_results['F1-Score'].max() - all_results['F1-Score'].min():.4f}")

# Type comparison
classical_avg = all_results[all_results['Type'] == 'Classical']['F1-Score'].mean()
dl_avg = all_results[all_results['Type'] == 'Deep Learning']['F1-Score'].mean()
print(f"\n5️⃣ AVERAGE PERFORMANCE BY TYPE:")
print(f"   Classical ML average: {classical_avg:.4f}")
print(f"   Deep Learning average: {dl_avg:.4f}")
print(f"   Difference: {dl_avg - classical_avg:.4f}")

print("\n" + "="*90)

### 4.2 Production Recommendations

In [ ]:
print("\n" + "="*90)
print("PRODUCTION DEPLOYMENT RECOMMENDATIONS")
print("="*90)

print("\n🚀 USE CASE 1: REAL-TIME APPLICATIONS (< 100ms latency)")
print("   Recommended: Logistic Regression or SVM")
print("   Reason: Fast inference, good accuracy (~88-89%)")
print("   Example: Live chat sentiment analysis, real-time feedback")

print("\n📊 USE CASE 2: BATCH PROCESSING (prioritize accuracy)")
print("   Recommended: DistilBERT")
print("   Reason: Highest accuracy (~92%), acceptable for batch jobs")
print("   Example: Analyzing thousands of reviews overnight")

print("\n⚖️ USE CASE 3: BALANCED APPROACH (medium latency, good accuracy)")
print("   Recommended: LSTM or SVM")
print("   Reason: Good balance of speed and performance")
print("   Example: API endpoints with moderate traffic")

print("\n💰 USE CASE 4: RESOURCE-CONSTRAINED (low memory/compute)")
print("   Recommended: Naive Bayes or Logistic Regression")
print("   Reason: Smallest model size, minimal compute requirements")
print("   Example: Edge devices, mobile applications")

print("\n🔍 USE CASE 5: INTERPRETABILITY REQUIRED")
print("   Recommended: Logistic Regression")
print("   Reason: Clear feature weights, explainable predictions")
print("   Example: Compliance-heavy industries, regulated applications")

print("\n" + "="*90)

## 5. Limitations and Challenges

In [ ]:
print("\n" + "="*90)
print("LIMITATIONS AND CHALLENGES IDENTIFIED")
print("="*90)

limitations = [
    ("1. Binary Classification Only",
     "Models only predict positive/negative, missing neutral sentiment.",
     "Solution: Extend to 3-class or 5-class classification."),
    
    ("2. Sarcasm Detection",
     "All models struggle with sarcastic reviews (e.g., 'Oh great, another masterpiece')",
     "Solution: Add sarcasm-specific features or specialized models."),
    
    ("3. Domain Specificity",
     "Trained only on movie reviews, may not generalize to other domains.",
     "Solution: Domain adaptation or training on multi-domain data."),
    
    ("4. Short Review Performance",
     "Reviews < 50 words show higher error rates due to limited context.",
     "Solution: Ensemble with character-level models for short texts."),
    
    ("5. Computational Cost (Deep Learning)",
     "BERT models require significant GPU memory and training time.",
     "Solution: Model distillation, quantization, or use TinyBERT."),
    
    ("6. No Aspect-Based Analysis",
     "Models don't identify WHAT is positive/negative (acting, plot, etc.)",
     "Solution: Implement aspect-based sentiment analysis (ABSA)."),
    
    ("7. Static Models",
     "No continuous learning from new reviews or changing language patterns.",
     "Solution: Implement online learning or periodic retraining pipeline."),
    
    ("8. Class Imbalance Sensitivity",
     "While our dataset is balanced, real-world data often isn't.",
     "Solution: Use SMOTE, class weighting, or focal loss.")
]

for title, problem, solution in limitations:
    print(f"\n{title}")
    print(f"   Problem: {problem}")
    print(f"   {solution}")

print("\n" + "="*90)

## 6. Future Work and Improvements

In [ ]:
print("\n" + "="*90)
print("FUTURE WORK AND RECOMMENDED IMPROVEMENTS")
print("="*90)

future_work = [
    ("🎯 Short-term Improvements (1-2 weeks)", [
        "Implement LIME/SHAP for model explainability",
        "Create confusion matrix analysis for error patterns",
        "Add cross-validation for all models",
        "Build simple web interface (Streamlit/Gradio)",
        "Add more preprocessing variations (stemming vs lemmatization)"
    ]),
    
    ("🚀 Medium-term Enhancements (1 month)", [
        "Ensemble methods (voting classifier with top 3 models)",
        "Hyperparameter optimization with Optuna/Ray Tune",
        "Deploy models as REST API (Flask/FastAPI)",
        "Implement A/B testing framework",
        "Add support for multi-class sentiment (1-5 stars)",
        "Create Docker containers for deployment"
    ]),
    
    ("🔬 Advanced Research (2-3 months)", [
        "Experiment with GPT-based models (GPT-3.5/GPT-4 fine-tuning)",
        "Implement aspect-based sentiment analysis",
        "Multi-lingual sentiment analysis (mBERT, XLM-R)",
        "Attention mechanism visualization",
        "Semi-supervised learning with unlabeled data",
        "Active learning for continuous improvement"
    ]),
    
    ("🏢 Production-Ready Features (ongoing)", [
        "Model monitoring and drift detection",
        "Automated retraining pipeline",
        "Load testing and performance optimization",
        "Cloud deployment (AWS SageMaker, GCP AI Platform)",
        "CI/CD pipeline with GitHub Actions",
        "Comprehensive logging and alerting system"
    ])
]

for category, items in future_work:
    print(f"\n{category}:")
    for i, item in enumerate(items, 1):
        print(f"   {i}. {item}")

print("\n" + "="*90)

## 7. Final Summary and Statistics

In [ ]:
print("\n" + "="*90)
print("PROJECT FINAL SUMMARY")
print("="*90)

print("\n📊 DATASET:")
print("   • Total reviews: 50,000 (25k train, 25k test)")
print("   • Classes: Binary (Positive/Negative)")
print("   • Balance: Perfectly balanced (50-50)")
print("   • Source: IMDb Movie Reviews")

print("\n🤖 MODELS EVALUATED:")
print("   Classical ML:")
print("   • Logistic Regression")
print("   • Naive Bayes")
print("   • Support Vector Machine (SVM)")
print("   • Random Forest")
print("   Deep Learning:")
print("   • Bidirectional LSTM (custom)")
print("   • DistilBERT (fine-tuned)")

print("\n🏆 TOP PERFORMERS:")
for i in range(min(3, len(ranked))):
    model = ranked.iloc[i]
    print(f"   {i+1}. {model['Model']}: {model['F1-Score']:.4f} F1-Score")

print("\n💡 KEY TAKEAWAYS:")
print("   ✓ Deep learning (DistilBERT) achieves best accuracy (~92%)")
print("   ✓ Classical ML (SVM) offers excellent speed-accuracy trade-off")
print("   ✓ All models achieve > 85% accuracy on this dataset")
print("   ✓ Choice depends on use case: latency vs accuracy requirements")
print("   ✓ Preprocessing quality significantly impacts performance")

print("\n🎯 PROJECT OBJECTIVES MET:")
print("   ✅ Implemented 6 different sentiment analysis models")
print("   ✅ Compared classical ML vs deep learning approaches")
print("   ✅ Achieved production-ready accuracy (>85%)")
print("   ✅ Comprehensive evaluation with multiple metrics")
print("   ✅ Identified limitations and future improvements")
print("   ✅ Created reusable, well-documented codebase")

print("\n📁 DELIVERABLES:")
print("   • 4 complete Jupyter notebooks")
print("   • 6 trained and saved models")
print("   • Professional visualizations and charts")
print("   • Comprehensive documentation (README)")
print("   • Results and insights analysis")

print("\n" + "="*90)
print("🎉 PROJECT COMPLETE - READY FOR SUBMISSION!")
print("="*90)

and viola!
**Project Statistics**:
- 50,000 reviews processed
- 6 models trained and evaluated
- 4 comprehensive notebooks
- 92% best accuracy achieved
- 100% learning accomplished! 🎓